In [ ]:
from mylibraries import *
X_train = pd.read_csv('X_train_scaled.csv')
X_test = pd.read_csv('X_test_scaled.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Získanie názvov stĺpcov z pôvodného X_train
column_names = X_train.columns

In [ ]:
from mylibraries import *
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, precision_recall_curve, average_precision_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.model_selection import learning_curve


In [ ]:
# Definovanie modelu Support Vector Machine (SVM)
from sklearn.svm import SVC
svm_model = SVC(C=10.0, gamma='auto', kernel='rbf', probability=True)

# Trénovanie modelu
svm_model.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred = svm_model.predict(X_test)

# Vyhodnotenie modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Testovacia presnosť: {accuracy:.4f}")

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

# Výpis klasifikačnej správy
print("Klasifikačná správa:")
print(classification_report(y_test, y_pred))

# Matica zámen
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matica zámen:")
print(conf_matrix)

# ROC krivka
y_pred_proba = svm_model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, label='ROC krivka (AUC = {:.2f})'.format(roc_auc_score(y_test, y_pred_proba)))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC krivka')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall krivka
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
average_precision = average_precision_score(y_test, y_pred_proba)
plt.figure()
plt.step(recall, precision, where='post', color='b', alpha=0.2, label=f'Priemerná presnosť = {average_precision:.2f}')
plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Krivka')
plt.legend(loc="lower left")
plt.show()

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(svm_model, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
plt.figure()
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Trénovacia presnosť')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Validačná presnosť')
plt.xlabel('Počet trénovacích príkladov')
plt.ylabel('Presnosť')
plt.title('Learning Curve')
plt.legend(loc="best")
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

# Vypočítať permutačnú dôležitosť atribútov
result = permutation_importance(svm_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

# Zobraziť dôležitosť atribútov
importances = result.importances_mean
indices = np.argsort(importances)[::-1]

print("Najdôležitejšie atribúty:")
for f in range(X_train.shape[1]):
    print(f"{f+1}. Atribút '{X_train.columns[indices[f]]}' s dôležitosťou {importances[indices[f]]}")

# Zobrazenie permutačnej dôležitosti atribútov vo forme grafu
plt.figure(figsize=(10, 6))
plt.bar(range(X_train.shape[1]), importances[indices], color='skyblue', align='center')
plt.xticks(range(X_train.shape[1]), X_train.columns[indices], rotation=90)
plt.title('Permutačná dôležitosť atribútov')
plt.xlabel('Atribúty')
plt.ylabel('Dôležitosť')
plt.show()


In [ ]:
# Zobrazenie dôležitosti atribútov
importances = result.importances_mean
indices = np.argsort(importances)[::-1]

# Získanie indexov najdôležitejších atribútov
top_n = 10
top_indices = indices[:top_n]

# Získanie dát len pre najdôležitejšie atribúty
X_train_top = X_train.iloc[:, top_indices]
X_test_top = X_test.iloc[:, top_indices]

# Natrénovanie modelu SVM len s najdôležitejšími atribútmi
svm_model_top = SVC(kernel='linear', C=1.0)
svm_model_top.fit(X_train_top, y_train)

# Predikcia na testovacích dátach
y_pred_svm_top = svm_model_top.predict(X_test_top)

# Vyhodnotenie modelu SVM s najdôležitejšími atribútmi
accuracy_svm_top = accuracy_score(y_test, y_pred_svm_top)
print(f"Testovacia presnosť modelu SVM s najdôležitejšími atribútmi: {accuracy_svm_top:.4f}")
# Klasifikačná správa modelu SVM s najdôležitejšími atribútmi
print("Klasifikačná správa modelu SVM s najdôležitejšími atribútmi:")
print(classification_report(y_test, y_pred_svm_top))

# Matica zámen modelu SVM s najdôležitejšími atribútmi

print("Matica zámen modelu SVM s najdôležitejšími atribútmi:")
print(confusion_matrix(y_test, y_pred_svm_top))

In [ ]:
# Inicializácia SVM modelu
svm_model = SVC(C=10.0, gamma='auto', kernel='rbf')

# Inicializácia Bagging Classifier s SVM ako základným modelom
from sklearn.ensemble import BaggingClassifier
bagging_svm_model = BaggingClassifier(estimator=svm_model, n_estimators=10, random_state=42)

# Trénovanie Bagging Classifier modelu
bagging_svm_model.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred = bagging_svm_model.predict(X_test)

# Vyhodnotenie modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Testovacia presnosť: {accuracy:.4f}")

# Výpis klasifikačnej správy
print("Klasifikačná správa:")
print(classification_report(y_test, y_pred))

# Matica zámen
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matica zámen:")
print(conf_matrix)